In [1]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark import SparkConf
from pyspark.sql import SparkSession
# from pyspark.sql.functions import *
import os
# conf = SparkConf()
# conf = conf.set("spark.executor.cores", 4)
# conf = conf.set("spark.executor.memory", "16g")
# conf = conf.set("spark.executor.instances", 10)
# conf = conf.set('spark.yarn.executor.memoryOverhead', '10g')
# conf = conf.set('spark.io.compression.codec', 'snappy')
# conf = conf.set("spark.sql.execution.arrow.enabled","true")

In [2]:
conf = SparkConf()
appName="Model Assessment: Churn Prediction Model"

In [3]:
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config('spark.dynamicAllocation.enabled', 'false') \
    .config('spark.executor.instances', '5') \
    .config('spark.executor.cores', '10') \
    .config('spark.executor.memory', '50g') \
    .config('spark.yarn.executor.memoryOverhead', '40g') \
    .config('spark.io.compression.codec', 'snappy') \
    .config('spark.sql.autoBroadcastJoinThreshold','0') \
    .enableHiveSupport() \
    .getOrCreate()

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.IllegalStateException: Spark context stopped while waiting for backend
	at org.apache.spark.scheduler.TaskSchedulerImpl.waitBackendReady(TaskSchedulerImpl.scala:835)
	at org.apache.spark.scheduler.TaskSchedulerImpl.postStartHook(TaskSchedulerImpl.scala:199)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:571)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j

In [ ]:
spark

In [ ]:
sdf_last_date_per_month = spark.sql("""select 
                                substring(max(updated_date),1 ,10) as last_date,
                                month(updated_date),    
                                year(updated_date)
                            from datamart.dm_churn_prediction_non_angsuran 
                           group by month(updated_date), year(updated_date)""")
sdf_last_date_per_month.show()
sdf_last_date_per_month.registerTempTable('tbl_last_date')

In [17]:
df_last_date = sdf_last_date_per_month.toPandas()
df_last_date

,last_date,month(CAST(updated_date AS DATE)),year(CAST(updated_date AS DATE))
0,2021-11-30,11,2021
1,2021-04-14,4,2021
2,2021-08-31,8,2021
3,2021-03-30,3,2021
4,2021-02-25,2,2021
5,2021-10-31,10,2021
6,2021-09-28,9,2021
7,2021-12-18,12,2021


In [63]:
sdfa = spark.sql("""

SELECT 
--    CONCAT(id,updated_date) c_key,
    CASE
        WHEN a.churn_month <= '2021-08-31' THEN 
            CASE WHEN c.churn_pred = TRUE THEN c.churn_month END
        ELSE a.churn_month
    END churn_month, 
    CASE
        WHEN a.churn_month <= '2021-08-31' THEN 
            CASE WHEN c.churn_pred = TRUE THEN c.churn_pred END
        ELSE a.churn_pred
    END churn_pred, 
    a.monetary, 
    a.kuadran, 
    a.id, 
    a.updated_date
FROM datamart.dm_churn_prediction_non_angsuran a
INNER JOIN tbl_last_date  b
on a.updated_date = b.last_date
LEFT JOIN datamart.dm_churn_prediction_dashboard c
on a.id = c.id


""").persist()

sdfa.registerTempTable("stage_0a")

In [61]:
sdfa.filter("""updated_date = '2021-08-31'""").groupby('id').agg({'id':'count'}).count()

KeyboardInterrupt: 

In [64]:
sdf_prediction_all = list()
# pov_date = '2021-08-31'

for idx, row in df_last_date.iterrows():
    pov_date = row['last_date']
    print(row['last_date'])
    sdfb = spark.sql("""

    SELECT 
    --    c.c_key,
        c.id,
        c.monetary, 
        c.kuadran, 
        c.updated_date,
        c.churn_pred, 
        CASE
            WHEN (c.churn_month < t_all.last_transaction_date = TRUE) THEN FALSE    -- not churn
            WHEN (c.churn_month < t_all.last_transaction_date = FALSE) THEN TRUE    -- churn
            WHEN (t.churn_month < t_all.last_transaction_date = TRUE) THEN FALSE    -- not churn
            WHEN (t.churn_month < t_all.last_transaction_date = FALSE) THEN TRUE    -- churn
        END AS actual,
        CASE
            WHEN c.churn_pred = TRUE AND (c.churn_month < t_all.last_transaction_date = TRUE) THEN 'TP'
            WHEN c.churn_pred = TRUE AND (c.churn_month < t_all.last_transaction_date = FALSE) THEN 'FP'
            WHEN c.churn_pred = FALSE AND (t.churn_month < t_all.last_transaction_date = TRUE) THEN 'TN'
            WHEN c.churn_pred = FALSE AND (t.churn_month < t_all.last_transaction_date = FALSE) THEN 'FN'
        END AS conclusion,
    --   c.churn_month,
    --    t.cif,
        SUM(t.total_up) AS total_up 
    FROM stage_0a c
    LEFT JOIN (SELECT cust.no_id,  last_day((max(tgl_transaksi) + interval 120 days)) as churn_month, sum(up) as total_up
                FROM datalake.pgd_tbl_transaksi_kredit kre
                LEFT JOIN (SELECT no_id, cif FROM 
                datalake.pgd_tbl_customer) cust
                ON kre.cif = cust.cif
                WHERE
                tgl_transaksi <= '{pov_date}' 
                AND kode_transaksi in ('0002',
                '0010',
                '0012',
                '0502',
                '2013',
                '2014',
                '2015',
                '2016',
                '2018',
                '2513',
                '2514',
                '2519',
                '2610',
                '2614',
                '2615',
                '2624',
                '30002',
                '30502',
                '32013',
                '32513',
                '32514')
                GROUP BY cust.no_id
                ) t
    ON c.id= t.no_Id
    LEFT JOIN (SELECT cust.no_id,  max(tgl_transaksi) as last_transaction_date, sum(up) as total_up
                FROM datalake.pgd_tbl_transaksi_kredit kre
                LEFT JOIN (SELECT no_id, cif FROM 
                datalake.pgd_tbl_customer) cust
                ON kre.cif = cust.cif
                WHERE
                kode_transaksi in ('0002',
                '0010',
                '0012',
                '0502',
                '2013',
                '2014',
                '2015',
                '2016',
                '2018',
                '2513',
                '2514',
                '2519',
                '2610',
                '2614',
                '2615',
                '2624',
                '30002',
                '30502',
                '32013',
                '32513',
                '32514')
                GROUP BY cust.no_id
                ) t_all
    ON c.id= t_all.no_Id
    WHERE c.updated_date = '{pov_date}' AND (c.churn_month < CURRENT_DATE OR t.churn_month < CURRENT_DATE)
    GROUP BY
    --    c.c_key,
        c.id,
        CASE
            WHEN (c.churn_month < t_all.last_transaction_date = TRUE) THEN FALSE    -- not churn
            WHEN (c.churn_month < t_all.last_transaction_date = FALSE) THEN TRUE    -- churn
            WHEN (t.churn_month < t_all.last_transaction_date = TRUE) THEN FALSE    -- not churn
            WHEN (t.churn_month < t_all.last_transaction_date = FALSE) THEN TRUE    -- churn
        END,
        CASE
            WHEN c.churn_pred = TRUE AND (c.churn_month < t_all.last_transaction_date = TRUE) THEN 'TP'
            WHEN c.churn_pred = TRUE AND (c.churn_month < t_all.last_transaction_date = FALSE) THEN 'FP'
            WHEN c.churn_pred = FALSE AND (t.churn_month < t_all.last_transaction_date = TRUE) THEN 'TN'
            WHEN c.churn_pred = FALSE AND (t.churn_month < t_all.last_transaction_date = FALSE) THEN 'FN'
        END,
    --    c.churn_month,
        c.churn_pred, 
        c.monetary, 
        c.kuadran,
    --    t.cif,
        c.updated_date 

    """.format(pov_date=pov_date)).persist()
    sdf_prediction_all.append(sdfb)



2021-11-30
2021-04-14
2021-08-31
2021-03-30
2021-02-25
2021-10-31
2021-09-28
2021-12-18


+--------+------+----------+------------+
|count(1)|actual|churn_pred|updated_date|
+--------+------+----------+------------+
|  458673|  true|      true|  2021-12-18|
|  887521|  true|     false|  2021-12-18|
+--------+------+----------+------------+



In [65]:
from functools import reduce
from pyspark.sql import DataFrame

dfs = sdf_prediction_all
df_merge = reduce(DataFrame.unionAll, dfs)

In [11]:
df_merge.show()

+----------------+---------+--------------------+------------+----------+------+----------+---------+
|              id| monetary|             kuadran|updated_date|churn_pred|actual|conclusion| total_up|
+----------------+---------+--------------------+------------+----------+------+----------+---------+
|0201041009720001|  1058100|     Prepare to Lose|  2021-11-30|      true|  true|        FP| 500000.0|
|0250026211890005| 27938200|Low Cost Maintenance|  2021-11-30|     false|  true|        FN|   2.54E7|
|0950034208860011|141608000|Sustained Relatio...|  2021-11-30|     false|  true|        FN| 1.3839E8|
|0950076104500267| 33140800|Sustained Relatio...|  2021-11-30|     false|  true|        FN|9100000.0|
|0951035201610105|455923600|Sustained Relatio...|  2021-11-30|     false|  true|        FN| 3.9645E8|
|0951035408700584|  3446000|     Prepare to Lose|  2021-11-30|      true|  true|        FP|3250000.0|
|0952014212825523|  9357400|Low Cost Maintenance|  2021-11-30|     false|  true|  

In [66]:
df_merge.registerTempTable("stage1")
spark.sql("""

SELECT count(*), sum(monetary), actual, churn_pred, conclusion
FROM stage1
GROUP BY
actual, churn_pred, conclusion, id

""").persist().show()

+--------+-------------+------+----------+----------+
|count(1)|sum(monetary)|actual|churn_pred|conclusion|
+--------+-------------+------+----------+----------+
|       3|     34597500|  true|      true|        FP|
|       4|     26296000|  true|     false|        FN|
|       4|     36587200|  true|      true|        FP|
|       4|    715275600|  true|     false|        FN|
|       4|    174302000|  true|     false|        FN|
|       4|     56700000|  true|     false|        FN|
|       3|     68314800|  true|     false|        FN|
|       3|     28854000|  true|     false|        FN|
|       3|      3105900|  true|      true|        FP|
|       4|    275158800|  true|      true|        FP|
|       4|    188047600|  true|     false|        FN|
|       4|     85311600|  true|     false|        FN|
|       4|     83740000|  true|     false|        FN|
|       3|      9768000|  true|     false|        FN|
|       4|    260178800|  true|     false|        FN|
|       4|     48392800|  tr

accuracy 	(TP+TN)/(ALL)
precision	TP/(TP+FP)
recall	    TP/(TP+FN)
f1 score	(2 x pre x recall) / (pre + recall)
specificity	TN/(TN+FP)

In [67]:
def get_accuracy(TP, TN, FP, FN):
    try:    
        metric = (TP+TN)/(TP+TN+FP+FN)
    except Exception as e:
        print(e)
        metric = 0
    finally:
        return metric*100
                 
def get_precision(TP, TN, FP, FN):
    try:
        metric =   TP/(TP+FP)
    except Exception as e:
        print(e)
        metric = 0
    finally:
        return metric*100
                 
def get_recall(TP, TN, FP, FN):
    try:
        metric =   TP/(TP+FN)
    except Exception as e:
        print(e)
        metric = 0
    finally:
        return metric*100
                 
def get_f1_score(TP, TN, FP, FN):
    try:    
        metric =   (2 * (TP/(TP+FP)) * (TP/(TP+FN))) / ((TP/(TP+FP)) + (TP/(TP+FN)))
    except Exception as e:
        print(e)
        metric = 0
    finally:
        return metric*100
                 
def get_specificity(TP, TN, FP, FN):
    try:
        metric =  TN/(TN+FP)      
    except Exception as e:
        print(e)
        metric = 0
    finally:
        return metric*100

def get_all_metrics(TP, TN, FP, FN):
    accuracy = get_accuracy(TP, TN, FP, FN)
    precision = get_precision(TP, TN, FP, FN)
    recall = get_recall(TP, TN, FP, FN)
    f1 = get_f1_score(TP, TN, FP, FN)
    specificity = get_specificity(TP, TN, FP, FN)
    
    return 'Accuracy:', accuracy, 'Precision:', precision, 'Recall:', recall, 'F1:', f1, 'Specificity:', specificity

In [68]:
# for sdf in sdf_prediction_all:

for idx, row in df_last_date.sort_values('last_date').iterrows():
    df = df_merge.filter("""updated_date = '""" + row['last_date'] + """'""").groupby('updated_date','conclusion').agg({'conclusion':'count'}).toPandas()
    df.columns = ['updated_date', 'conclusion', 'count']
    TP = int(df.loc[df['conclusion']=='TP','count']) if len(df.loc[df['conclusion']=='TP','count'])>0 else 0
    TN = int(df.loc[df['conclusion']=='TN','count']) if len(df.loc[df['conclusion']=='TN','count'])>0 else 0
    FN = int(df.loc[df['conclusion']=='FN','count']) if len(df.loc[df['conclusion']=='FN','count'])>0 else 0
    FP = int(df.loc[df['conclusion']=='FP','count']) if len(df.loc[df['conclusion']=='FP','count'])>0 else 0
    print('Metric at ', row['last_date'], "-------------------")
    print(get_all_metrics(TP, TN, FP, FN))

Metric at  2021-02-25 -------------------
division by zero
division by zero
division by zero
division by zero
division by zero
('Accuracy:', 0, 'Precision:', 0, 'Recall:', 0, 'F1:', 0, 'Specificity:', 0)
Metric at  2021-03-30 -------------------
('Accuracy:', 35.272500180145556, 'Precision:', 35.272500180145556, 'Recall:', 100.0, 'F1:', 52.15028942789161, 'Specificity:', 0.0)
Metric at  2021-04-14 -------------------
('Accuracy:', 37.35115925619266, 'Precision:', 37.35115925619266, 'Recall:', 100.0, 'F1:', 54.387832557749064, 'Specificity:', 0.0)
Metric at  2021-08-31 -------------------
('Accuracy:', 23.049881031489143, 'Precision:', 8.98619799424089, 'Recall:', 3.2745956507580414, 'F1:', 4.800042431314309, 'Specificity:', 51.79341537814243)
Metric at  2021-09-28 -------------------
('Accuracy:', 18.472396991447265, 'Precision:', 5.207228408851901, 'Recall:', 2.109115725073574, 'F1:', 3.0022227276689932, 'Specificity:', 42.83531119327722)
Metric at  2021-10-31 -------------------
('Ac

In [14]:
#int(df.loc[df['conclusion']=='TP','count']) if len(df.loc[df['conclusion']=='TP','count'])>0 else 0

In [24]:
pdf_merge = df_merge.select(['actual','churn_pred']).toPandas()

In [25]:
pdf_merge

,actual,churn_pred
0,True,True
1,True,False
2,True,False
3,True,False
4,True,False
...,...,...
10306406,True,True
10306407,True,False
10306408,True,False
10306409,True,False


In [33]:
from sklearn.metrics import classification_report
#target_names = ['True', 'False']
dict={"True":1, "False":0}
#pdf_merge.column.replace(dict, inplace=True)
pdf_merge['churn_pred','actual'].astype("int")
#print(classification_report(int(pdf_merge['churn_pred']), int(pdf_merge['actual'])))

AttributeError: 'DataFrame' object has no attribute 'select'

In [47]:
#pdf_merge['churn_pred'].astype("int")
#pdf_merge['actual'].astype("int")
type(pdf_merge['actual'])

pandas.core.series.Series

In [22]:
from sklearn.metrics import confusion_matrix
dict={"True":1, "False":0}
df.column.replace(dict, inplace=True)
confusion_matrix(df_merge['churn_pred'], df_merge['actual'])

TypeError: object of type 'Column' has no len()

In [ ]:
df_merge.groupby('conclusion').agg({'conclusion':'count'}).show()

In [21]:
df_merge

DataFrame[id: string, monetary: bigint, kuadran: string, updated_date: string, churn_pred: boolean, actual: boolean, conclusion: string, total_up: double]

In [ ]:
plot_auc_roc(df_merge['churn_pred'], df_merge['actual'])